In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot  as plt

# Device configuration
device = 'cpu'#torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 1 Этап - подготовка данных!

In [4]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [20]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

# 2 Этап - Нейронная сеть!

<img src='Lenet5.png'>

In [8]:
# Fully connected neural network with one hidden layer
class Lenet5(nn.Module):
    def __init__(self, hidden_size1=500, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=0)
        self.batchnorm1 = nn.BatchNorm2d(20)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1, padding=0)
        self.batchnorm2 = nn.BatchNorm2d(50)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(50 * 4 * 4, hidden_size1) # fully connected
        self.fc2 = nn.Linear(hidden_size1, num_classes)  
    
    def forward(self, x): # вход размера 1 x 28 x 28
        out = self.conv1(x) # на выходе размер 20 x 24 x 24
        out = self.batchnorm1(out)
        out = self.maxpool(out) # на выходе размер 20 x 12 x 12
        out = self.relu(out)
        out = self.conv2(out)  # на выходе размер 50 x 8 x 8
        out = self.batchnorm2(out)
        out = self.maxpool(out) # на выходе размер 50 x 4 x 4
        out = self.relu(out)
        out = out.reshape(-1, 50 * 4 * 4) # на выходе размер 800
        out = self.fc1(out) # на выходе размер 500
        out = self.relu(out)
        out = self.fc2(out) # на выходе размер 10
        
        return out

model = Lenet5(hidden_size1=500, num_classes=10).to(device)


In [ ]:
class NN_block(nn.Module):
    def __init__(self, input_size):
        self.fc = nn.Linear(input_size, input_size)
        self.activation = nn.ReLU()
    def forward(self, x):
        out = x
        out = self.fc(out)
        out = self.activation(out)
        return out

class NN(nn.Module):
    def __init__(self, input_size):
        self.block = NN_block(input_size)

    def forward(self, x):
        out = x
        for i in range(10):
            out = self.block(out)
        return out

In [9]:
model

Lenet5(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu): ReLU()
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# 3 этап - Loss-функция и оптимизатор

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# for p in model.parameters():
#     print(p)

In [13]:
model.conv1

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))

In [16]:
model.conv1.weight.shape

torch.Size([20, 1, 5, 5])

In [17]:
model.conv1.weight[0]

tensor([[[-0.1336,  0.1986,  0.1058, -0.0056, -0.1951],
         [-0.0087,  0.1795, -0.0947,  0.1587, -0.1071],
         [ 0.0484,  0.1796,  0.0676, -0.0872, -0.1053],
         [ 0.1274,  0.0026,  0.1132,  0.0337,  0.0309],
         [-0.0159,  0.0652,  0.0210, -0.1225, -0.1820]]],
       grad_fn=<SelectBackward>)

In [15]:
model.conv1.bias

Parameter containing:
tensor([ 0.0903, -0.0053, -0.0760, -0.0431,  0.0639, -0.1037,  0.0474,  0.0758,
         0.1286,  0.1351,  0.1329, -0.1138,  0.0712, -0.0653, -0.1147, -0.0645,
         0.1803, -0.1606,  0.1681,  0.0211], requires_grad=True)

# 4 этап - обучение нейросети

In [23]:
for i, (images, labels) in enumerate(train_loader):  
    break
images.shape, labels.shape

(torch.Size([100, 1, 28, 28]), torch.Size([100]))

In [24]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

C:\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/5], Step [100/600], Loss: 0.1504
Epoch [1/5], Step [200/600], Loss: 0.1995
Epoch [1/5], Step [300/600], Loss: 0.1652
Epoch [1/5], Step [400/600], Loss: 0.0907
Epoch [1/5], Step [500/600], Loss: 0.0755
Epoch [1/5], Step [600/600], Loss: 0.0759
Accuracy of the network on the 10000 test images: 98.49 %
Epoch [2/5], Step [100/600], Loss: 0.0134
Epoch [2/5], Step [200/600], Loss: 0.0809
Epoch [2/5], Step [300/600], Loss: 0.0315
Epoch [2/5], Step [400/600], Loss: 0.0262
Epoch [2/5], Step [500/600], Loss: 0.0125
Epoch [2/5], Step [600/600], Loss: 0.0315
Accuracy of the network on the 10000 test images: 98.8 %
Epoch [3/5], Step [100/600], Loss: 0.0375
Epoch [3/5], Step [200/600], Loss: 0.0221
Epoch [3/5], Step [300/600], Loss: 0.0064
Epoch [3/5], Step [400/600], Loss: 0.0193
Epoch [3/5], Step [500/600], Loss: 0.0034
Epoch [3/5], Step [600/600], Loss: 0.0392
Accuracy of the network on the 10000 test images: 98.71 %
Epoch [4/5], Step [100/600], Loss: 0.0150
Epoch [4/5], Step [200/600], 

In [28]:
ss = []
for name, p in model.named_parameters():
    ss.append(p.numel())
    print(name, p.shape, p.numel())
print(sum(ss))

conv1.weight torch.Size([20, 1, 5, 5]) 500
conv1.bias torch.Size([20]) 20
conv2.weight torch.Size([50, 20, 5, 5]) 25000
conv2.bias torch.Size([50]) 50
fc1.weight torch.Size([500, 800]) 400000
fc1.bias torch.Size([500]) 500
fc2.weight torch.Size([10, 500]) 5000
fc2.bias torch.Size([10]) 10
431080
